In [15]:
import pyvips
import os
import argparse
from pathlib import Path
from tqdm import tqdm_notebook as tqdm
import glob
from natsort import natsorted
import cv2 as cv
from hashlib import sha256
import shutil
import matplotlib.pyplot as plt
import numpy as np

In [16]:
# search_pattern = "butterfly/*.png"

# image_paths = natsorted(glob.glob("butterfly/*.png"))

In [17]:
def read_image(image_path):
    return cv.cvtColor(cv.cvtColor(cv.imread(image_path, 0), cv.COLOR_BAYER_BG2BGR), cv.COLOR_RGB2GRAY)

def line_prepender(filename, line):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(line.rstrip('\r\n') + '\n' + content)

In [36]:
!rm -rf leaves_and_lillies
new_dir = "leaves_and_lillies"
os.makedirs(new_dir)

In [37]:
# for image_path in tqdm(image_paths):
#     image = read_image(image_path).astype(np.int)
#     image = np.int0(image*255/150)
#     image = np.clip(image, 0, 255).astype(np.uint8)
#     cv.imwrite(os.path.join(new_dir, os.path.basename(image_path)), image)

## Do DZI conversion for each image

In [38]:
search_pattern = "leafs//*.tif"
base_dir = 'leaves_and_lillies'

In [39]:
# take first five for now as a test
dz_image_paths = natsorted(glob.glob(search_pattern))
os.makedirs(base_dir, exist_ok=True)
output_folders = []
for dzp in tqdm(dz_image_paths):
    image = pyvips.Image.new_from_file(dzp)
    base_name = os.path.basename(dzp)
    output_folder = os.path.splitext(base_name)[0]
    output_folder = os.path.join(base_dir, output_folder)
    output_folders.append(output_folder)
    os.makedirs(output_folder, exist_ok=False)
    tile_size = 512
    image.dzsave(output_folder, suffix='.jpg[Q=100]',depth=9, tile_width=tile_size, tile_height=tile_size)

/home/rapiduser/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [40]:
shutil.copy2("base.js", os.path.join(base_dir, "movie.js"))
shutil.copy2("index.html", os.path.join(base_dir, "index.html"))

'leaves_and_lillies/index.html'

In [41]:
tileSources = [os.path.basename(of) + ".dzi" for of in output_folders]

In [42]:
line = "tileSources = " + str(tileSources) + ";"

In [43]:
line_prepender(os.path.join(base_dir, "movie.js"), line)